# Clustering Toronto Neighborhood using K-Means Algorithm

Importing Essential Libraries

In [1]:
import numpy as np
import pandas as pd
import json 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
from bs4 import BeautifulSoup
print('Libraries are imported')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries are imported


# Part 1

Importing Data(Wikipedia Table)

In [2]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data_soup = BeautifulSoup(data,'html5lib')
data_soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"783e11df-a6ca-4ee4-b8b7-846b78e7b710","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

In [3]:
tables = data_soup.find_all('table')
tables

[<table cellpadding="2" cellspacing="0" rules="all" style="width:100%;">
 
 <tbody><tr>
 <td style="width:11%;">
 <p>M1A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>
 <td style="width:11%;">
 <p>M2A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>
 <td style="width:11%;">
 <p>M3A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
 </p>
 </td>
 <td style="width:11%;">
 <p>M4A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
 </p>
 </td>
 <td style="width:11%;">
 <p>M5A<br/><span style="font-size:85%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a> / <a href="/wiki/Harbourfront,_Toronto" title="Harbourfro

Creating Dataframe

In [4]:
table_contents=[]
table=data_soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.style

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Shape of the dataframe

In [6]:
df.shape

(103, 3)

# Part 2

Importing Data(Longitude-Latitude)

In [7]:
df_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging dataframe

In [8]:
df_lat_long.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_toronto = pd.merge(df,df_lat_long,on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Part 3

In [9]:
df_toronto = df_toronto[df_toronto['Borough'].str.contains(pat='Toronto')]
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Map of Toronto

In [10]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, long, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Applying K-Means Algorithm

In [11]:
kclusters = 5
toronto_grouped_clustering = df_toronto.drop(['PostalCode','Borough','Neighborhood'], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 0, 2, 0, 4, 3], dtype=int32)

Final Dataframe

In [12]:
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)
df_toronto

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,2,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,3,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


Map of Toronto after Clustering

In [13]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [14]:
df_toronto.loc[df_toronto['Cluster Labels'] == 0, df_toronto.columns[[1] + list(range(5, df_toronto.shape[1]))]]

,PostalCode,Longitude
2,M5A,-79.360636
9,M5B,-79.378937
15,M5C,-79.375418
20,M5E,-79.373306
24,M5G,-79.387383
30,M5H,-79.384568
36,M5J,-79.381752
42,M5K,-79.381576
48,M5L,-79.379817
87,M5V,-79.394420


In [15]:
df_toronto.loc[df_toronto['Cluster Labels'] == 1, df_toronto.columns[[1] + list(range(5, df_toronto.shape[1]))]]

,PostalCode,Longitude
61,M4N,-79.388790
62,M5N,-79.416936
67,M4P,-79.390197
68,M5P,-79.411307
73,M4R,-79.405678
79,M4S,-79.388790
83,M4T,-79.383160
86,M4V,-79.400049


In [16]:
df_toronto.loc[df_toronto['Cluster Labels'] == 2, df_toronto.columns[[1] + list(range(5, df_toronto.shape[1]))]]

,PostalCode,Longitude
25,M6G,-79.422564
37,M6J,-79.419750
43,M6K,-79.428191
74,M5R,-79.405678
80,M5S,-79.400049
84,M5T,-79.400049


In [17]:
df_toronto.loc[df_toronto['Cluster Labels'] == 3, df_toronto.columns[[1] + list(range(5, df_toronto.shape[1]))]]

,PostalCode,Longitude
19,M4E,-79.293031
35,M4J,-79.338106
41,M4K,-79.352188
47,M4L,-79.315572
54,M4M,-79.340923
100,M7Y,-79.321558


In [18]:
df_toronto.loc[df_toronto['Cluster Labels'] == 4, df_toronto.columns[[1] + list(range(5, df_toronto.shape[1]))]]

,PostalCode,Longitude
31,M6H,-79.442259
69,M6P,-79.464763
75,M6R,-79.456325
81,M6S,-79.484450
